In [1]:
#This sets up a t-test to test propagations between clean and perturbed models. Note that this only works right now because we have both the clean and perturbed model. We won't be able to to this test when we only have the perturbed model

import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
sns.set()


In [2]:
input_fn = './../models/160M/160M_exp2_0_perturb/propagation_inputs.csv'
original_fn = './../models/160M/160M_exp2_0_clean/scores.csv'
perturbed_fn = './../models/160M/160M_exp2_0_perturb/scores.csv'

In [3]:
inputs = pd.read_csv(input_fn)
print(inputs.columns)

Index(['example_index', 'text', 'sub_index', 'original', 'synonym',
       'substituted?'],
      dtype='object')


In [4]:
print(inputs.head(5))

   example_index                                               text  \
0         920792  I also tried passing the parameters directly t...   
1         930635  Tag Archives: Marcilhac\n\nCombined Yoga & Can...   
2         716255  Recent Posts\n\nThanks so far. I guess it's go...   
3         385756  28 June 2012\n\nBook du jour - ball of words\n...   
4         976260  It was our weekly routine. A ceremony practise...   

   sub_index original synonym  substituted?  
0        276     nice    good         False  
1      62494     nice    good          True  
2         41     nice    good          True  
3        823     nice    good         False  
4      41774     nice    good         False  


In [5]:
#loads the scored files
original = pd.read_csv(original_fn, header=None)
original.columns = ['example_idx', 'w1_prob', 'w2_prob', 'w1_rank', 'w2_rank']

perturbed = pd.read_csv(perturbed_fn, header=None)
perturbed.columns = ['example_idx', 'w1_prob', 'w2_prob', 'w1_rank', 'w2_rank']

In [8]:
#Only analyze the examples that were perturbed
original = original[inputs["substituted?"]]
perturbed = perturbed[inputs["substituted?"]]

In [7]:
print(inputs["original"].unique())

['nice' 'size' 'way' 'small' 'guy' 'start' 'just' 'first' 'next' 'permit'
 'business' 'monitor' 'big' 'more' 'trouble' 'return' 'lead' 'very'
 'totally' 'help' 'area' 'idea' 'device' 'equal' 'subject' 'man' 'own'
 'choose' 'house' 'file' 'try' 'provide' 'analysis' 'fell' 'full' 'move'
 'cut' 'many' 'perhaps' 'guess' 'accept' 'say' 'grow' 'voice' 'team']


In [26]:
out_file = []
for pair in inputs["original"].unique():
    #mask for analyzing a single perturbation example "pair"
    mask = inputs["original"] == pair
    temp_original = original["w2_prob"][mask] - original["w1_prob"][mask]
    temp_perturbed = perturbed["w2_prob"][mask] - perturbed["w1_prob"][mask]

    # print(temp_original.values.mean())
    # print(temp_perturbed.values.mean())
    temp_arr = [pair]
    result = stats.ttest_rel(10000 * temp_original.values, 10000 * temp_perturbed.values)
    temp_arr += [result.statistic, result.pvalue]
    out_file += [temp_arr]



In [28]:
df = pd.DataFrame(out_file, columns=["word_pair", "statistic", "pvalue"])

In [30]:
df.to_csv("./../models/160M/160M_exp2_0_perturb/t-test.csv")